# Problem 1: PatchCamelyon
*Course*: DS807 \
*Authors*: August E. Wennerwald, Kasper Lin Hannberg, Oliver Klejst, Søren Pico, Thomas Fischer

## Modules and data

In [4]:
# -- MODULES -- #

# Tensorflow
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50

# Additional
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
import random
from tqdm import tqdm

# -- DIRECTORY -- # 
wd = './'

In [2]:
# NOTE: un-comment if run in google colab

# from google.colab import drive
# drive.mount('/content/drive')
# wd = /content/drive/My Drive/DS807 (AML)
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [17]:
# -- LOAD DATA -- # 

# AE, VAE
d1a, d2a, d3a = tfds.load('patch_camelyon', split=[f'train[50%:]',f'test[50%:]',f'validation[50%:]'],
                          data_dir=wd+'DATA/PCAM',
                          download=False,
                          shuffle_files=True)

# Classification 
d1b, d2b, d3b = tfds.load('patch_camelyon', split=[f'train[:50%]',f'test[:50%]',f'validation[:50%]'],
                          data_dir=wd+'DATA/PCAM',
                          download=False,
                          shuffle_files=True)


# Convert data to (x,y)-format 
def convert_sample(sample):
    image, label = sample['image'], sample['label']
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = tf.one_hot(label, 2, dtype=tf.float32)
    return image, label

# AE, VAE
train_images_a = d1a.map(lambda x: convert_sample(x)[0]).batch(128)
validation_images_a = d3a.map(lambda x: convert_sample(x)[0]).batch(128)
test_images_a = d2a.map(lambda x: convert_sample(x)[0]).batch(128)

# Classification 
train_data_b = d1b.map(convert_sample).batch(64)
validation_data_b = d3b.map(convert_sample).batch(64)
test_data_b = d2b.map(convert_sample).batch(64)

In [36]:
# -- ADDITIONAL -- #

# Function for freezing parameters of input model 
def freeze_model_parameters(model): 
    for l in model.layers: 
        l.trainable = False

# Function for generating plots of traning history 
def plot_training_hist(hist):
  fig = plt.figure(figsize=plt.figaspect(0.1 * 2))

  ax = fig.add_subplot(1, 2, 1)
  ax.plot(hist.history['val_loss'], label='Validation loss')
  ax.plot(hist.history['loss'], label='Training loss')


  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()

  ax = fig.add_subplot(1, 2, 2)
  ax.plot(hist.history['val_accuracy'], label='Validation accuracy')
  ax.plot(hist.history['accuracy'], label='Training accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend()

## Baseline CNN models

In [ ]:
# NOTE Søren kode

## Autoender (AE)

### Building and training AE-model

In [ ]:
# NOTE Søren kode 

### AE-based classification

## Variational autoencoder (VAE)

### Building and training VAE-models

In [20]:
# -- GENERAL SETUP OF VAE MODELS -- #

# VAE class
class VAE(tf.keras.Model):
    def __init__(self, latent_dim, encoder, decoder):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = encoder
        self.decoder = decoder

    def encode(self, x):
        params = self.encoder(x)
        return tf.split(params, num_or_size_splits=2, axis=1) # mean, logvar

    def decode(self, z):
        return self.decoder(z)

    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * 0.5) + mean

    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(100, self.latent_dim)) 
        return tf.sigmoid(self.decode(eps))

# Loss function  
def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    vals = -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi)

    return tf.reduce_sum(vals, axis=raxis)

def compute_loss(model, x):
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    x_logit = model.decode(z)
    cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
    logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
    logpz = log_normal_pdf(z, 0., 0.)
    logqz_x = log_normal_pdf(z, mean, logvar)

    return -tf.reduce_mean(logpx_z + logpz - logqz_x)

# Training step 
@tf.function
def train_step(model, x, optimizer):
    with tf.GradientTape() as tape:
        loss = compute_loss(model, x)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# Reconstruct and plot test/validation images 
def generate_and_show_images(model, epoch, test_sample):
    mean, logvar = model.encode(test_sample)
    z = model.reparameterize(mean, logvar)
    predictions = model.sample(z)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.show()

# Setup optimizer 
optimizer = tf.keras.optimizers.legacy.Adam(1e-4)   # Use when run on M1/M2 
#optimizer = tf.keras.optimizers.Adam(1e-4)          # Use otherwise 

# Sample for reconstrunction 
test_sample = next(iter(validation_images_a.take(1)))[:16]


In [16]:
# -- VAE_16 MODEL -- #

vae16_encoder = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, kernel_size= 3, strides = 1, padding='same', activation='relu', input_shape=(96, 96, 3)),
    tf.keras.layers.Conv2D(128, kernel_size=3, strides= 2, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(256, kernel_size=3, strides= 2, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2 * 16), # latent dim = 16
])

vae16_decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=24*24*32, activation='relu', input_shape=(16,)),
    tf.keras.layers.Reshape(target_shape= (24, 24, 32)),
    tf.keras.layers.Conv2DTranspose(256, kernel_size = 3, strides= 2, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(128, kernel_size = 3, strides= 1, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(64, kernel_size = 3, strides= 1, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(32, kernel_size = 3, strides= 2, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(3,3, strides= 1, padding="same", activation="sigmoid")
])

#vae16_encoder.summary()
#vae16_decoder.summary()

vae16 = VAE(16, vae16_encoder, vae16_decoder)


In [ ]:
# -- TRAINING OF VAE_16 MODEL -- #
for epoch in range(30):
    for train_x in train_images_a:
        train_step(vae16, train_x, optimizer)

    loss = tf.keras.metrics.Mean()
    for test_x in test_images_a:
        loss(compute_loss(vae16, test_x))
    variational_lower_bound = -loss.result()

    print(f'Epoch: {epoch+1}, Test set variational lower bound: {variational_lower_bound}')
    generate_and_show_images(vae16, epoch, test_sample)

In [22]:
# -- SAVE VAE_16 EN+DECODER WEIGHTS -- #
#vae16_encoder.save_weights(wd+'saved_weights_VAE/VAE_16_encoder_weights.h5')
#vae16_decoder.save_weights(wd+'saved_weights_VAE/VAE_16_decoder_weights.h5')

In [23]:
# -- VAE_32 MODEL -- #

vae32_encoder = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, kernel_size= 3, strides = 1, padding='same', activation='relu', input_shape=(96, 96, 3)),
    tf.keras.layers.Conv2D(128, kernel_size=3, strides= 2, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(256, kernel_size=3, strides= 2, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2 * 32), # latent dim = 32
])

vae32_decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=24*24*32, activation='relu', input_shape=(32,)),
    tf.keras.layers.Reshape(target_shape= (24, 24, 32)),
    tf.keras.layers.Conv2DTranspose(256, kernel_size = 3, strides= 2, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(128, kernel_size = 3, strides= 1, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(64, kernel_size = 3, strides= 1, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(32, kernel_size = 3, strides= 2, padding='same', activation='relu'),
    tf.keras.layers.Conv2DTranspose(3,3, strides= 1, padding="same", activation="sigmoid")
])

#vae32_encoder.summary()
#vae32_decoder.summary()

vae32 = VAE(32, vae32_encoder, vae32_decoder)


In [ ]:
# -- TRAINING OF VAE_32 MODEL -- #
for epoch in range(40):
    for train_x in train_images_a:
        train_step(vae32, train_x, optimizer)

    loss = tf.keras.metrics.Mean()
    for test_x in test_images_a:
        loss(compute_loss(vae32, test_x))
    variational_lower_bound = -loss.result()

    print(f'Epoch: {epoch+1}, Test set variational lower bound: {variational_lower_bound}')
    generate_and_show_images(vae32, epoch, test_sample)

In [ ]:
# -- SAVE VAE_32 EN+DECODER WEIGHTS -- #
#vae16_encoder.save_weights(wd+'saved_weights_VAE/VAE_32_encoder_weights.h5')
#vae16_decoder.save_weights(wd+'saved_weights_VAE/VAE_32_decoder_weights.h5')

In [ ]:
# -- FUNCTIONS FOR REPORT PLOTS -- # 

# Plot sample reconstruction 
def sample_VAE_reconstruction(model, idx = None):    
    fig = plt.figure(figsize=(10,10))
    
    batch = next(iter(test_images_a))
    if idx is None:
        idx = random.randint(0,len(batch))
    print(idx)
    
    original_img = batch[idx].numpy()
    fig.add_subplot(1,2,1)
    plt.imshow(original_img)
    plt.axis('off')

    original_img_reshaped = original_img.reshape((1,)+original_img.shape)
    
    mean, log_var = model.encode(original_img_reshaped)
    z = model.reparameterize(mean, log_var)
    reconstructed_img = model.sample(z)
    reconstructed_img_reshaped = np.reshape(reconstructed_img, (96,96,3))
    fig.add_subplot(1,2,2)
    plt.imshow(reconstructed_img_reshaped)
    plt.axis('off')


# Plot sample VAE_16 reconstruction 
vae16_encoder.load_weights(wd+'saved_weights_VAE/VAE_16_encoder_weights.h5')
vae16_decoder.load_weights(wd+'saved_weights_VAE/VAE_16_decoder_weights.h5')
vae16_pre_trained = VAE(16, vae16_encoder, vae16_decoder)
sample_VAE_reconstruction(vae16_pre_trained)

### VAE-based classification

In [35]:
# -- SET UP VAE-BASED CLASSIFIERS -- #

# Freeze encoder parameters 
freeze_model_parameters(vae16_encoder)
freeze_model_parameters(vae32_encoder)

# Build classifiers 

# VAE_16 classifier 
vae16_classifier = tf.keras.models.Sequential([
    vae16_encoder,
    tf.keras.layers.Reshape((32,1)),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=2, activation='softmax')
])

# VAE_32 classifier 
vae32_classifier = tf.keras.models.Sequential([
    vae32_encoder,
    tf.keras.layers.Reshape((64,1)),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=2, activation='softmax')
])

#vae16_classifier.summary()
#vae32_classifier.summary()

In [40]:
# -- SET UP TRAINING CALLBACK AND COMPILE CLASSIFIERS -- #

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                           patience = 3,
                                                           restore_best_weights=True)

vae16_classifier.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

vae32_classifier.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# -- TRAIN VAE_16 CLASSIFIER -- #

vae_16_tr_hist = vae16_classifier.fit(train_data_b,
                                    validation_data=validation_data_b,
                                    epochs=30,
                                    callbacks=[early_stopping_callback])

vae16_classifier.save_weights(wd+'saved_weights_VAE/VAE_16_classifier_weights.h5')

In [ ]:
# -- TRAIN VAE_32 CLASSIFIER -- #

vae_32_tr_hist = vae32_classifier.fit(train_data_b,
                                    validation_data=validation_data_b,
                                    epochs=30,
                                    callbacks=[early_stopping_callback])

vae32_classifier.save_weights(wd+'saved_weights_VAE/VAE_32_classifier_weights.h5')

In [ ]:
# -- EVALUATE CLASSIFIER AND PLOT TRAINING -- #

vae16_classifier.evaluate(test_data_b)
vae32_classifier.evaluate(test_data_b)

plot_training_hist(vae_16_tr_hist)
plot_training_hist(vae_32_tr_hist)

## Transfer learning

### EfficientNet

In [ ]:
# NOTE Søren kode

### ResNet50

In [41]:
# -- SETUP RESNET50 -- #

# Load w/o standard input-output layers 
pretrained_ResNet50 = ResNet50(weights='imagenet', 
                       include_top = False, 
                       input_shape=(96,96,3))

In [44]:
# -- APPROACH 1 -- #
# Freeze entire ResNet50 such that only fully connected part is trainable 

ResNet50_1 = pretrained_ResNet50
freeze_model_parameters(ResNet50_1)

# ResNet50_1.summary()

PCam_ResNet50_1 = tf.keras.models.Sequential()
PCam_ResNet50_1.add(ResNet50_1)

# Add layers for binary classification 
PCam_ResNet50_1.add(tf.keras.layers.Flatten())
PCam_ResNet50_1.add(tf.keras.layers.Dense(250, activation='relu')),
PCam_ResNet50_1.add(tf.keras.layers.Dense(2, activation='softmax'))

# PCam_ResNet50_1.summary()

# Compile 
PCam_ResNet50_1.compile(optimizer='adam', 
                      loss = 'categorical_crossentropy', 
                      metrics=['accuracy'])

In [45]:
# -- SET UP TRAINING CALLBACK -- #
ResNet50_early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                           patience=5,
                                                           restore_best_weights=True)

In [ ]:
# -- TRAIN PCam_ResNet50_1 -- #
PCam_ResNet50_1_tr_hist = PCam_ResNet50_1.fit(train_data_b, 
                         validation_data=validation_data_b,
                         epochs=50, 
                         callbacks=[early_stopping_callback])

PCam_ResNet50_1.save_weights(wd+'ResNet50_weights/PCam_ResNet50_1_weights.h5')

In [ ]:
# -- EVALUATE PCam_ResNet50_1 AND PLOT TRAINING -- #
PCam_ResNet50_1.evaluate(test_data_b)

plot_training_hist(PCam_ResNet50_1_tr_hist)